# Create Output folder, copy files to there, and Comine them if needed

- The combine script takes about 2 hours to run

In [ ]:
# Tell the notebook where to find the pipeline functions
import sys
sys.path.append('/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Lesion_Pipeline/dependencies/')

In [ ]:
import os
import shutil
from glob import glob 
import GNMPipeline as p

# Add your inputs

In [ ]:
input_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test/"  #Folder with input files, can use the folder created by Combine script previously
input_type='BIDS' #'BIDS' or 'Folder'
output_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test_output/"
                     

IMAGE_TYPES = ['t1', 'T2', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='t1' #your ideal registration target, case sensitive
Reg_target_2='T2'

# Make dictionary of subjects and sessions

In [ ]:
subject_sessions=p.create_input_dict(input_dir, subjects_to_skip=subjects_to_skip, input_type=input_type)

# Visualize output folder system

In [ ]:
print(f'Your output folder, {output_dir}, will look something like this: ')
p.print_tree(subject_sessions)

# Create output folder system

In [ ]:
# Create output folders for each subject and session based on the subject_sessions dictionary
for subject, sessions in subject_sessions.items():
    
    subject_folder = os.path.join(output_dir, subject)
    
    # Create the subject folder if it doesn't exist
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    
    
    # Loop through each session for the subject
    for session in sorted(set(sessions)):
        print(f'Preparing output folders for {session} for {subject}')
        session_folder = os.path.join(subject_folder, session)
         
        # Check if the session folder exists and is not empty
        if os.path.exists(session_folder):
            if os.listdir(session_folder):
                print(f"{subject}: {session_folder} exists and is not empty")
        else:
            # Create the session folder if it doesn't exist
            if not os.path.exists(session_folder):
                os.makedirs(session_folder)

# Copy files to output directory, combining files as needed

In [ ]:
# Copy images to output folders; Combine images as needed
combine_dict={}
for subject, sessions in subject_sessions.items():
    subject_folder = os.path.join(output_dir, subject)
    

    # Loop through each session for the subject
    for session in sorted(set(sessions)):
        session_folder = os.path.join(subject_folder, session)
        
        if not os.path.exists(f'{session_folder}/SELECTED'):
            os.makedirs(f'{session_folder}/SELECTED')
         
        # Loop through each image type 
        for image_type in IMAGE_TYPES:

            # Check if a file with the image type already exists in the session folder
            if glob(f'{session_folder}/SELECTED/*{image_type}*.nii*'):
                    print(f"Error: A file with image type '{image_type}' already exists in {session_folder}/SELECTED")
            else: 
                # Get the list of images for the image type
                if input_type == 'Folder':
                    images = glob(f'{input_dir}/{subject}*{session}*{image_type}*.nii*') 
                elif input_type == 'BIDS':
                    images = glob(f'{input_dir}/{subject}/{session}/*{image_type}*.nii*')
             
                # Check if there are more than 3 images to combine
                if len(images) > 3:
                    print(f"Error: More than 3 images found for participant {subject}, session {session}, and image type {image_type}.")
                    continue
                
                # Combine the images if there are more than 1
                if len(images) > 1 and image_type not in ['FLAIR', 'flair', 'Flair']:
                    
                    #Copy the un-combined images to the output folder so you have them
                    os.makedirs(f'{session_folder}/SELECTED/precombine', exist_ok=True)
                    [shutil.copy(i,f'{session_folder}/SELECTED/precombine') for i in images]
                    
                    #create the combine command and submit it to SLURM
                    ##### MAKE SEPARATE COMBINE FOLDERS IF >1 TYPE PER SUBJECT NEEDS IT, OTHERWISE CLEAN UP WILL DELETE STUFF BEFORE ITS DONE
                    print(f"combining: {images}")
                    images_to_combine=glob(f'{session_folder}/SELECTED/precombine/*{image_type}*nii*') 
                    command = p.combine_images(f'{session_folder}/SELECTED', images_to_combine, f'{subject}_{session}_{image_type}', clean_up=True)
                    job_name = f"combine_images_{subject}_{session}_{image_type}"
                    job_id=p.submit_slurm_job(job_name, command)
                    combine_dict[(subject,session)] = job_id
                    
                if len(images) > 1 and image_type in ['FLAIR', 'flair', 'Flair']:
                    print('You have more than one FLAIR image, Combining these may not work well, please select one')
                
                # Copy the single image to the session folder
                elif len(images) == 1:
                    print(f"copying: {images} to {session_folder}/SELECTED/{subject}_{session}_{image_type}.nii.gz")
                    shutil.copy(images[0], f'{session_folder}/SELECTED/{subject}_{session}_{image_type}.nii.gz')

# Print the number of combine jobs submitted to SLURM
if len(combine_dict) > 0:
    print('You have', len(combine_dict), 'Combine Jobs submitted to SLURM; subject and job IDs are stored in combine_dict')
    print('You can type "squeue -u $USER" into your terminal to track SLURM job progress')
    print('You can check the output file matching the jobid in combine_dict to see code outputs and any errors')
else:
    print('You have', len(combine_dict), 'Combine Jobs')

# Check your submitted jobs

In [ ]:
print("These are the jobs that should have been submitted:")
print(combine_dict)

In [ ]:
!squeue -u $USER 